In [9]:
import sys
sys.path.append("../python")

import json
import h5py
import numpy as np
import arviz as az
import compute_costs
import evaluations

In [10]:
settings = [
    "MH",
    "MLDA - meshes",
    "MLDA - electrodes",
    "MLDA - source models",
    "MLDA"
]

parameters_paths = [
    "../configs/test_mh/mh.json",
    "../configs/test_mlda/mlda_meshes.json",
    "../configs/test_mlda/mlda_electrodes.json",
    "../configs/test_mlda/mlda_sourcemodels.json"
    "../configs/test_mlda/mlda.json"]

samples = []
chains = []
num_samples = []
burn_in = []
costs = []
    
for path in parameters_paths:
    file = open(path)
    config = json.load(file)
    file.close()

    num_samples.append(config["Sampling"]["NumSamples"])
    burn_in.append(config["Sampling"]["BurnIn"])
    chains.append(config["Setup"]["Chains"])
    
    samples_mlda = []
    for c in range(config["Setup"]["Chains"]):
        sample_path = config["Setup"]["OutputPath"] + config["Sampling"]["ResultFile"] 
        fin = h5py.File(sample_path + "_" + str(c) + ".h5")
        samples_mlda.append(np.array(fin['/samples']))
        fin.close()
    samples.append(samples_mlda)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/anne/Masterarbeit/masterarbeit/2d/results/test_mlda/mlda_0.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [3]:
# compute ess
ess = []

for sample in samples:
    ess_mlda = []
    for chain in sample: 
        if(evaluations.is_converged(chain)==False): 
            ess_mlda.append("Not converged")  
        elif(evaluations.burn_in_check(chain)==False): 
            ess_mlda.append("Burn-in too small")   
        else:
            ess_mlda.append(min(az.ess(chain[0]),az.ess(chain[1]),az.ess(chain[2])))
    ess.append(ess_mlda)



In [4]:
print("Samples (on finest level) per effective sample \n")


for j in range(min(chains)):
    for i in range(len(settings)):
        if(ess[i][j]=="Not converged"):
            print(settings[i] + ": Not converged!")
        elif(ess[i][j]=="Burn-in too small"):
            print(settings[i] + ": Burn-in too small!")  
        else:
            print(settings[i] + ": " + str((num_samples[i]-burn_in[i])/ess[i][j]))
            
    print("\n")

Samples (on finest level) per effective sample 

MH: 319.79274703656637
MLDA - meshes: Not converged!
MLDA - source models: 98.17081313280832
MLDA - electrodes: 83.96572690829464


MH: 273.5824942637429
MLDA - meshes: 215.52146975198787
MLDA - source models: Not converged!
MLDA - electrodes: 79.4903642255413


MH: Burn-in too small!
MLDA - meshes: 270.70764998559383
MLDA - source models: 148.17662055599513
MLDA - electrodes: 212.51178475484053




In [11]:
costs = np.zeros(len(parameters_paths))
for i in range(len(settings)):
    costs[i] = evaluations.costs_per_sample(parameters_paths[i])

Dipole:
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
volume_conductor.time_creating_grid = 0.257831
volume_conductor.time_reordering_labels = 0.0038806
volume_conductor.time = 0.261713
time_element_neighborhood_map 0.0215895 s
degree = 1
element_type = hexahedron
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
sigma = 0.11508410052537331
{'Level3_V': {0: array([-8.59176596, -8.65824627, -8.11953982, -8.82336024, -9.05078691,
       -8.17118861, -8.11743752, -9.40115755, -7.83388808, -9.07572896,
       -7.78199894, -9.21043606, -9.20101762, -7.74062238, -8.99684312,
       -8.36669191, -8.02116365, -7.49394186, -6.3020461 , -4.10238027,
       -0.08874167,  5.87401153, 13.59164013, 20.39407033, 23.02747444,
       21.82225669, 19.34628697, 16.483797  , 13.93158746, 11.8399258 ,
       10.11433934,  2.77875358,  2.2505126 ,  1.38875621, -0.12522059,
        0.17385677])}}
volume_conductor.time_creating_grid = 0.35705
volume_conductor.time_reordering_labels = 0.01012

In [13]:
print(costs)
print("Costs per effective sample \n")

for j in range(min(chains)):
    for i in range(len(settings)):
        if(ess[i][j]=="Not converged"):
            print(settings[i] + ": Not converged!")
        elif(ess[i][j]=="Burn-in too small"):
            print(settings[i] + ": Burn-in too small!")  
        else:
            print(settings[i] + ": " + str(costs[i]/ess[i][j]))
            
    print("\n")

[209.07938957 420.41724443]
Costs per effective sample 

MH: 1.857279787222339
MLDA: Not converged!


MH: 1.588901691619106
MLDA: 9.537783413639293


MH: Burn-in too small!
MLDA: 11.980017289920387


